<a href="https://www.kaggle.com/code/mahmoud17/xgboost-lgbm-dataquest-challenge?scriptVersionId=128317312" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/temp-xgb/xgboostlgbm.csv
/kaggle/input/temp-xgb/xgboostlgbmada.csv
/kaggle/input/temp-xgb/xgboost(2).csv
/kaggle/input/dataquest-challenge-2/SampleSubmission.csv
/kaggle/input/dataquest-challenge-2/train.csv
/kaggle/input/dataquest-challenge-2/test.csv


In [2]:
train_data = pd.read_csv('/kaggle/input/dataquest-challenge-2/train.csv')
test_data = pd.read_csv('/kaggle/input/dataquest-challenge-2/test.csv')

In [3]:
sample_submission = pd.read_csv('/kaggle/input/dataquest-challenge-2/SampleSubmission.csv')
sample_submission.columns

Index(['ABSTRACT', 'Analysis of PDEs', 'Applications',
       'Artificial Intelligence', 'Astrophysics of Galaxies',
       'Computation and Language', 'Computer Science',
       'Computer Vision and Pattern Recognition',
       'Cosmology and Nongalactic Astrophysics',
       'Data Structures and Algorithms', 'Differential Geometry',
       'Earth and Planetary Astrophysics', 'Fluid Dynamics',
       'Information Theory', 'Instrumentation and Methods for Astrophysics',
       'Machine Learning', 'Materials Science', 'Mathematics', 'Methodology',
       'Number Theory', 'Optimization and Control', 'Physics',
       'Representation Theory', 'Robotics', 'Social and Information Networks',
       'Statistics', 'Statistics Theory', 'Strongly Correlated Electrons',
       'Superconductivity', 'Systems and Control'],
      dtype='object')

In [4]:
train_data.head()

,ABSTRACT,category
0,a backpressure algorithm has been widely used ...,", Computer Science, Mathematics, Optimization ..."
1,"inside this paper, we present an exactly solva...",", Physics, Strongly Correlated Electrons"
2,a present numerical study aims at shedding lig...,", Physics, Fluid Dynamics"
3,with the help of angle-resolved photoemission ...,", Physics, Strongly Correlated Electrons, Supe..."
4,based on sub-arcsecond atacama large millimete...,", Physics, Astrophysics of Galaxies"


In [5]:
main_categories = ['Analysis of PDEs', 'Applications',
       'Artificial Intelligence', 'Astrophysics of Galaxies',
       'Computation and Language', 'Computer Science',
       'Computer Vision and Pattern Recognition',
       'Cosmology and Nongalactic Astrophysics',
       'Data Structures and Algorithms', 'Differential Geometry',
       'Earth and Planetary Astrophysics', 'Fluid Dynamics',
       'Information Theory', 'Instrumentation and Methods for Astrophysics',
       'Machine Learning', 'Materials Science', 'Mathematics', 'Methodology',
       'Number Theory', 'Optimization and Control', 'Physics',
       'Representation Theory', 'Robotics', 'Social and Information Networks',
       'Statistics', 'Statistics Theory', 'Strongly Correlated Electrons',
       'Superconductivity', 'Systems and Control']

In [6]:
def segment_tags(data):
    data = data.copy()
    data["tags"] = data["category"].apply(lambda s : [word.strip() for word in s.split(",") if word != ""])
    for cat in main_categories : 
        data[cat] = data["tags"].apply(lambda s : 1 if cat in s else 0)
    return data

In [7]:
data = segment_tags(train_data)

In [8]:
data.head()

,ABSTRACT,category,tags,Analysis of PDEs,Applications,Artificial Intelligence,Astrophysics of Galaxies,Computation and Language,Computer Science,Computer Vision and Pattern Recognition,...,Optimization and Control,Physics,Representation Theory,Robotics,Social and Information Networks,Statistics,Statistics Theory,Strongly Correlated Electrons,Superconductivity,Systems and Control
0,a backpressure algorithm has been widely used ...,", Computer Science, Mathematics, Optimization ...","[Computer Science, Mathematics, Optimization a...",0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,1
1,"inside this paper, we present an exactly solva...",", Physics, Strongly Correlated Electrons","[Physics, Strongly Correlated Electrons]",0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
2,a present numerical study aims at shedding lig...,", Physics, Fluid Dynamics","[Physics, Fluid Dynamics]",0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,with the help of angle-resolved photoemission ...,", Physics, Strongly Correlated Electrons, Supe...","[Physics, Strongly Correlated Electrons, Super...",0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,1,0
4,based on sub-arcsecond atacama large millimete...,", Physics, Astrophysics of Galaxies","[Physics, Astrophysics of Galaxies]",0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [9]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

# Download the stopwords and punkt tokenizer if not already downloaded
nltk.download('stopwords')
nltk.download('punkt')

# Clean the text data
stop_words = set(stopwords.words('english'))
porter = PorterStemmer()

def clean_text(text):
    # Tokenize the text into words
    words = word_tokenize(text.lower())
    
    # Remove stop words
    words = [word for word in words if word not in stop_words]
    
    # Stem the words
    words = [porter.stem(word) for word in words]
    
    # Join the words back into a single string
    cleaned_text = ' '.join(words)
    
    return cleaned_text

data['cleaned_text'] = data['ABSTRACT'].apply(clean_text)

# Convert the text into numerical vectors using TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectors = tfidf_vectorizer.fit_transform(data['cleaned_text'])


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
data.head()

,ABSTRACT,category,tags,Analysis of PDEs,Applications,Artificial Intelligence,Astrophysics of Galaxies,Computation and Language,Computer Science,Computer Vision and Pattern Recognition,...,Physics,Representation Theory,Robotics,Social and Information Networks,Statistics,Statistics Theory,Strongly Correlated Electrons,Superconductivity,Systems and Control,cleaned_text
0,a backpressure algorithm has been widely used ...,", Computer Science, Mathematics, Optimization ...","[Computer Science, Mathematics, Optimization a...",0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,backpressur algorithm wide use distribut solut...
1,"inside this paper, we present an exactly solva...",", Physics, Strongly Correlated Electrons","[Physics, Strongly Correlated Electrons]",0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,"insid paper , present exactli solvabl model co..."
2,a present numerical study aims at shedding lig...,", Physics, Fluid Dynamics","[Physics, Fluid Dynamics]",0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,present numer studi aim shed light mechan unde...
3,with the help of angle-resolved photoemission ...,", Physics, Strongly Correlated Electrons, Supe...","[Physics, Strongly Correlated Electrons, Super...",0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,help angle-resolv photoemiss spectroscopi reve...
4,based on sub-arcsecond atacama large millimete...,", Physics, Astrophysics of Galaxies","[Physics, Astrophysics of Galaxies]",0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,base sub-arcsecond atacama larg millimeter/sub...


In [11]:
data.describe()

,Analysis of PDEs,Applications,Artificial Intelligence,Astrophysics of Galaxies,Computation and Language,Computer Science,Computer Vision and Pattern Recognition,Cosmology and Nongalactic Astrophysics,Data Structures and Algorithms,Differential Geometry,...,Optimization and Control,Physics,Representation Theory,Robotics,Social and Information Networks,Statistics,Statistics Theory,Strongly Correlated Electrons,Superconductivity,Systems and Control
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,...,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,0.044800,0.041900,0.098300,0.042000,0.046700,0.418200,0.068600,0.044100,0.035200,0.037000,...,0.044300,0.278000,0.025600,0.067100,0.044500,0.271900,0.03850,0.063300,0.038300,0.039000
std,0.206875,0.200371,0.297735,0.200599,0.211006,0.493288,0.252785,0.205328,0.184294,0.188771,...,0.205771,0.448036,0.157947,0.250208,0.206213,0.444961,0.19241,0.243514,0.191929,0.193605
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000


# Modeling

In [12]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    data['cleaned_text'], data.drop(["ABSTRACT","category","tags"],axis=1), test_size=0.2, random_state=42)

# Convert the text into numerical vectors using TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X_train_vectors = tfidf_vectorizer.fit_transform(X_train)
X_test_vectors = tfidf_vectorizer.transform(X_test)

In [13]:
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score

target_features = main_categories

# Train and evaluate a model for each target feature using cross-validation and hyperparameter optimization
models = []
accuracies = []
for feature in target_features:
    print(feature + " : ")

    # Choose a machine learning algorithm (LightGBM in this case)
    clf = LGBMClassifier()

    # Train the model on the training data
    clf.fit(X_train_vectors, y_train[feature])

    # Evaluate the model on the test data
    y_pred = clf.predict_proba(X_test_vectors)[:,1]
    accuracy = roc_auc_score(y_test[feature], y_pred)

    # Save the best model and its score
    models.append(clf)
    accuracies.append(accuracy)

    print('Best ROC AUC for', feature, ':', accuracy)


Analysis of PDEs : 
Best ROC AUC for Analysis of PDEs : 0.9859814891559607
Applications : 
Best ROC AUC for Applications : 0.9073048042574907
Artificial Intelligence : 
Best ROC AUC for Artificial Intelligence : 0.8658882022870942
Astrophysics of Galaxies : 
Best ROC AUC for Astrophysics of Galaxies : 0.9913674013679088
Computation and Language : 
Best ROC AUC for Computation and Language : 0.9843393947680248
Computer Science : 
Best ROC AUC for Computer Science : 0.9234479940937402
Computer Vision and Pattern Recognition : 
Best ROC AUC for Computer Vision and Pattern Recognition : 0.9429666829097382
Cosmology and Nongalactic Astrophysics : 
Best ROC AUC for Cosmology and Nongalactic Astrophysics : 0.987066881333057
Data Structures and Algorithms : 
Best ROC AUC for Data Structures and Algorithms : 0.953600449515906
Differential Geometry : 
Best ROC AUC for Differential Geometry : 0.9962402088772846
Earth and Planetary Astrophysics : 
Best ROC AUC for Earth and Planetary Astrophysics 

In [14]:
np.mean(accuracies)

0.9594610078927323

In [15]:
import xgboost as xgb

target_features = main_categories
# Train and evaluate a model for each target feature using cross-validation and hyperparameter optimization
xgb_models = []
xgb_accuracies = []
for feature in target_features:
    print(feature + " : ")
    
    # Choose a machine learning algorithm (XGBoost in this case)
    clf = xgb.XGBClassifier(objective='binary:logistic')

    clf.fit(X_train_vectors, y_train[feature])


    best_model = clf
    best_score = roc_auc_score(y_test[feature],clf.predict_proba(X_test_vectors)[:,1])

    # Save the best model and its score
    xgb_models.append(best_model)
    xgb_accuracies.append(best_score)

    print('Best ROC AUC for', feature, ':', best_score)


Analysis of PDEs : 
Best ROC AUC for Analysis of PDEs : 0.980538748445918
Applications : 
Best ROC AUC for Applications : 0.8838849602682802
Artificial Intelligence : 
Best ROC AUC for Artificial Intelligence : 0.8477903260174728
Astrophysics of Galaxies : 
Best ROC AUC for Astrophysics of Galaxies : 0.9788611284965971
Computation and Language : 
Best ROC AUC for Computation and Language : 0.9807560842220399
Computer Science : 
Best ROC AUC for Computer Science : 0.9145292143794075
Computer Vision and Pattern Recognition : 
Best ROC AUC for Computer Vision and Pattern Recognition : 0.9507029491242837
Cosmology and Nongalactic Astrophysics : 
Best ROC AUC for Cosmology and Nongalactic Astrophysics : 0.9848883362823395
Data Structures and Algorithms : 
Best ROC AUC for Data Structures and Algorithms : 0.945272879207008
Differential Geometry : 
Best ROC AUC for Differential Geometry : 0.9941821532790662
Earth and Planetary Astrophysics : 
Best ROC AUC for Earth and Planetary Astrophysics 

In [16]:
np.mean(xgb_accuracies)

0.9501796522436968

In [17]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer

cv_scores_means = []

# Convert the text into numerical vectors using TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X_vectors = tfidf_vectorizer.fit_transform(data['cleaned_text'])

# Define a list of target feature names
target_features = main_categories

# Train and evaluate the best model for each target feature using cross-validation
for i, feature in enumerate(target_features):
    # Choose the best model
    best_model = models[i]
    
    # Evaluate the model using cross-validation
    cv_scores = cross_val_score(best_model, X_vectors, data[feature], cv=5, scoring='roc_auc')
    cv_scores_means.append(cv_scores.mean())
    
    # Print the results
    print('ROC AUC for', feature, ':', cv_scores.mean())
    
print(np.mean(cv_scores_means))

ROC AUC for Analysis of PDEs : 0.97932505113346
ROC AUC for Applications : 0.8933746934388465
ROC AUC for Artificial Intelligence : 0.8584560454604869
ROC AUC for Astrophysics of Galaxies : 0.9911845113828412
ROC AUC for Computation and Language : 0.9819014223235223
ROC AUC for Computer Science : 0.9274167330691144
ROC AUC for Computer Vision and Pattern Recognition : 0.9499086438665632
ROC AUC for Cosmology and Nongalactic Astrophysics : 0.9839463766332885
ROC AUC for Data Structures and Algorithms : 0.9575015793869035
ROC AUC for Differential Geometry : 0.9912183211248632
ROC AUC for Earth and Planetary Astrophysics : 0.9849632262766423
ROC AUC for Fluid Dynamics : 0.9794725586870126
ROC AUC for Information Theory : 0.9348225172354525
ROC AUC for Instrumentation and Methods for Astrophysics : 0.9684057322224943
ROC AUC for Machine Learning : 0.9291773456160713
ROC AUC for Materials Science : 0.9748599801770792
ROC AUC for Mathematics : 0.9532758104754977
ROC AUC for Methodology : 0.9

In [18]:
predictions_df = sample_submission.copy()

In [19]:
# Convert the text into numerical vectors using the same TfidfVectorizer used for training

X_vectors = tfidf_vectorizer.fit_transform(data['cleaned_text'])
test_data['cleaned_text'] = test_data['ABSTRACT'].apply(clean_text)
X_test_vectors = tfidf_vectorizer.transform(test_data['cleaned_text'])


# Make predictions for each target feature using the trained models
for i, feature in enumerate(target_features):
    lgbm_model = models[i]
    xgb_model = xgb_models[i]
    lgbm_model.fit(X_vectors,data[feature])
    xgb_model.fit(X_vectors,data[feature])
    y_pred_lgbm = lgbm_model.predict_proba(X_test_vectors)[:,1]
    y_pred_xgb = xgb_model.predict_proba(X_test_vectors)[:,1]
    predictions_df[feature] = (y_pred_lgbm + y_pred_xgb) / 2
predictions_df.head()

,ABSTRACT,Analysis of PDEs,Applications,Artificial Intelligence,Astrophysics of Galaxies,Computation and Language,Computer Science,Computer Vision and Pattern Recognition,Cosmology and Nongalactic Astrophysics,Data Structures and Algorithms,...,Optimization and Control,Physics,Representation Theory,Robotics,Social and Information Networks,Statistics,Statistics Theory,Strongly Correlated Electrons,Superconductivity,Systems and Control
0,this paper studies empirical risk minimization...,0.000983,0.030136,0.026687,0.000057,0.001080,0.639973,0.020386,0.000150,0.000989,...,0.006949,0.002322,0.000007,0.000565,0.000903,0.844854,0.043686,0.000074,0.000019,0.000402
1,orthogonal time frequency space (otfs) modulat...,0.000049,0.002497,0.026541,0.000029,0.000210,0.814165,0.015001,0.000023,0.001652,...,0.007724,0.006785,0.000061,0.012361,0.001424,0.056988,0.001104,0.000105,0.000108,0.030402
2,a intelligent transportation system (its) targ...,0.000030,0.340039,0.148504,0.000006,0.000100,0.714872,0.008429,0.000020,0.000155,...,0.002584,0.001991,0.000003,0.001947,0.010428,0.909948,0.001992,0.000026,0.000005,0.027389
3,event-based cameras have shown great promise i...,0.000040,0.000647,0.065043,0.000010,0.000084,0.930446,0.512124,0.000019,0.001299,...,0.000379,0.006883,0.000018,0.026739,0.000604,0.091950,0.000232,0.000018,0.000013,0.000237
4,we report on orbital-dependent quasiparticle d...,0.000176,0.000476,0.003304,0.000357,0.000146,0.011062,0.000859,0.000422,0.000142,...,0.000120,0.998907,0.000169,0.000111,0.000562,0.004748,0.000246,0.747496,0.963706,0.000141


In [20]:
predictions_df.to_csv('xgblgbm.csv', index=False)